In [1]:
num_topics = 5
num_words = 5
passes = 10
seed = 100

In [2]:
import pandas as pd
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation, strip_numeric
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import gensim

# Load the data
data = pd.read_csv(r"C:\Users\Philip\Downloads\merged_data.csv\merged_data.csv")

# Custom filter to remove punctuation and numeric characters
custom_filters = [lambda x: x.lower(), strip_punctuation, strip_numeric]

# Apply preprocessing to each document in the text column and keep track of the business ID
processed_docs = []
business_ids = []
id_to_name = {}
review_counts = {}
for doc, business_id, business_name in zip(data['text'], data['business_id'], data['name']):
    processed_docs.append(preprocess_string(doc, custom_filters))
    business_ids.append(business_id)
    id_to_name[business_id] = business_name  # Populate the mapping dictionary

# Create a dictionary representation of the documents
dictionary = Dictionary(processed_docs)

# Filter out extremes to remove too rare or too common words
dictionary.filter_extremes(no_below=15)

# Convert document into the bag-of-words (BoW) format
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Training the LDA model
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10, random_state=seed)

# Get topic distribution for each document
doc_topics = [lda_model.get_document_topics(bow) for bow in corpus]

# Creating a DataFrame to hold the topic distribution for each business ID
topic_df = pd.DataFrame(0, index=set(business_ids), columns=['Topic_' + str(i) for i in range(num_topics)])

# Aggregate the topic distributions for each business ID
for business_id, doc_topic in zip(business_ids, doc_topics):
    for topic, prop in doc_topic:
        topic_df.at[business_id, 'Topic_' + str(topic)] += prop

# Count the number of reviews for each business ID
for business_id in business_ids:
    review_counts[business_id] = review_counts.get(business_id, 0) + 1

# Normalize the topic distributions for each business ID by the number of reviews
for business_id in topic_df.index:
    topic_df.loc[business_id] /= review_counts[business_id]

# Replace the index of the DataFrame with business names using the mapping dictionary
topic_df = topic_df.rename(index=id_to_name)

print(topic_df)

                                         Topic_0   Topic_1   Topic_2  \
Scarlet Lane Brewing Gastropub          0.080439  0.320441  0.113862   
Books & Brews - South Indy              0.116516  0.430242  0.083199   
Fermentery Form                         0.022564  0.709548  0.079311   
Xfinity Live! Philadelphia              0.305724  0.414195  0.098622   
Manayunk Brewery                        0.236262  0.184962  0.155378   
...                                          ...       ...       ...   
18th Street Brewery - Indy Taproom      0.120281  0.344823  0.142872   
Ellison Brewing Company - Indianapolis  0.111152  0.232465  0.176528   
William Street Common                   0.310817  0.180672  0.117899   
Parks On Tap                            0.203454  0.477032  0.077276   
Kuma's Corner                           0.189930  0.124644  0.114296   

                                         Topic_3   Topic_4  
Scarlet Lane Brewing Gastropub          0.310439  0.166278  
Books & Brews

In [3]:
topics = lda_model.print_topics(num_words=10)
topics

[(0,
  '0.022*"that" + 0.021*"t" + 0.016*"not" + 0.013*"they" + 0.013*"my" + 0.012*"this" + 0.010*"at" + 0.010*"have" + 0.009*"be" + 0.009*"you"'),
 (1,
  '0.017*"you" + 0.017*"beer" + 0.016*"that" + 0.016*"they" + 0.014*"on" + 0.013*"s" + 0.010*"have" + 0.010*"their" + 0.009*"this" + 0.009*"beers"'),
 (2,
  '0.087*"we" + 0.039*"were" + 0.033*"our" + 0.023*"had" + 0.016*"us" + 0.011*"on" + 0.011*"my" + 0.010*"back" + 0.010*"very" + 0.010*"at"'),
 (3,
  '0.028*"great" + 0.022*"food" + 0.022*"place" + 0.021*"beer" + 0.018*"good" + 0.017*"have" + 0.016*"are" + 0.016*"this" + 0.016*"they" + 0.015*"you"'),
 (4,
  '0.016*"had" + 0.014*"my" + 0.014*"good" + 0.014*"were" + 0.013*"burger" + 0.011*"on" + 0.011*"cheese" + 0.010*"fries" + 0.010*"that" + 0.009*"which"')]

In [4]:
topic_names = {
    0: "Customer Views",
    1: "Beer Selection",
    2: "Group Experience",
    3: "Ambience",
    4: "Burger"
}
# Example of renaming columns in a DataFrame
topic_df.rename(columns=lambda x: topic_names[int(x.split('_')[1])], inplace=True)
topic_df.to_csv(r"C:\Users\Philip\Downloads\LDA_output.csv")

In [5]:
topic_df

,Customer Views,Beer Selection,Group Experience,Ambience,Burger
Scarlet Lane Brewing Gastropub,0.080439,0.320441,0.113862,0.310439,0.166278
Books & Brews - South Indy,0.116516,0.430242,0.083199,0.262827,0.100893
Fermentery Form,0.022564,0.709548,0.079311,0.168760,0.011916
Xfinity Live! Philadelphia,0.305724,0.414195,0.098622,0.142984,0.031237
Manayunk Brewery,0.236262,0.184962,0.155378,0.275072,0.141521
...,...,...,...,...,...
18th Street Brewery - Indy Taproom,0.120281,0.344823,0.142872,0.234471,0.150816
Ellison Brewing Company - Indianapolis,0.111152,0.232465,0.176528,0.239371,0.235089
William Street Common,0.310817,0.180672,0.117899,0.155878,0.230268
Parks On Tap,0.203454,0.477032,0.077276,0.182107,0.053494
